# Poisson Noise and Anscombe Transform Denoising

This notebook simulates Poisson noise (photon counting noise) in images and removes it using the **Anscombe Transform method**.

## What This Notebook Does:

1. **Adds Poisson Noise**: Simulates noise at 5 different intensity levels (scales: 0.1, 0.5, 1.0, 2.0, 5.0)
2. **Applies Anscombe Transform Denoising**: Uses a 3-step process:
   - Transform the noisy image to stabilize variance
   - Apply a denoising filter (Gaussian, Median, or Bilateral)
   - Inverse transform back to original domain
3. **Calculates Quality Metrics**: PSNR, SSIM, MSE, Entropy, and more
4. **Generates Visualizations**: Creates histograms and sample image comparisons

## Output Directory Structure:
```
data_results/data/
├── Poisson_Anscombe_Metric/    # CSV files with quality metrics
├── Poisson_Noised_Images/       # Images with noise added
├── Poisson_Anscombe_Sample/     # Side-by-side comparisons of original, noisy, denoised
└── graph/poisson/               # Histogram visualizations of metrics
```

## Applications:
- Medical imaging (X-ray, CT scans)
- Astronomy (telescope images)
- Low-light photography
- Any photon-limited imaging scenario

## 1. Import Required Libraries

In [8]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set random seeds for reproducibility
random.seed("This is a seed.")
np.random.seed(42)

# Configure matplotlib for nice-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully")
print(f'✓ Random seed set for reproducibility')

✓ All libraries imported successfully
✓ Random seed set for reproducibility


## 2. Configure Input and Output Paths

**IMPORTANT**: Update `base_path` to point to your folder containing original images.

The notebook will automatically create the output directory structure if it doesn't exist.

In [10]:
# ============ INPUT PATH ============
# Update this to point to your images folder
base_path = "../Photos_Subset/Original"

# ============ OUTPUT PATH ============
# All results will be saved under this directory
output_base = "data_results/data"

# Create output directory if it doesn't exist
Path(output_base).mkdir(parents=True, exist_ok=True)
print(f"✓ Output directory: {output_base}")

# Verify input path exists
if not os.path.exists(base_path):
    raise FileNotFoundError(f"Input folder not found at: {base_path}")

if not os.path.isdir(base_path):
    raise NotADirectoryError(f"{base_path} is not a directory")

print(f"✓ Input path verified: {base_path}")

✓ Output directory: data_results/data
✓ Input path verified: ../Photos_Subset/Original


## 3. Define Poisson Noise Function

Poisson noise models the randomness in photon counting. This is the primary noise source in:
- Low-light imaging
- Medical X-rays
- Astronomy

The `scale` parameter controls noise intensity:
- **Lower scale** (e.g., 0.1) = more noise (fewer photons)
- **Higher scale** (e.g., 5.0) = less noise (more photons)

In [11]:
def add_poisson_noise(image, scale=1.0):
    """
    Add Poisson noise to an image.
    
    Parameters:
    -----------
    image : numpy array
        Input image (BGR format from cv2)
    scale : float
        Noise intensity control
        - scale < 1.0: more noise (lower photon count)
        - scale = 1.0: standard Poisson noise
        - scale > 1.0: less noise (higher photon count)
    
    Returns:
    --------
    numpy array : Image with Poisson noise added
    """
    # Normalize to [0, 1] range
    image_normalized = image.astype(np.float64) / 255.0
    
    # Scale to control noise level
    scaled_image = image_normalized / scale
    
    # Apply Poisson noise (models photon counting)
    noisy = np.random.poisson(scaled_image * 255) / 255.0
    
    # Scale back
    noisy = noisy * scale
    
    # Convert back to uint8 [0, 255]
    noisy = np.clip(noisy * 255, 0, 255).astype(np.uint8)
    
    return noisy

print("✓ Poisson noise function defined")

✓ Poisson noise function defined


## 4. Define Anscombe Transform Functions

The **Anscombe Transform** converts Poisson-distributed noise into approximately Gaussian noise with stable variance. This makes standard denoising filters more effective.

**Three-step denoising process:**
1. Apply Anscombe transform: `y = 2√(x + 3/8)`
2. Apply denoising filter (Gaussian, Median, or Bilateral)
3. Apply inverse transform: `x = (y/2)² - 3/8`

In [12]:
def anscombe_transform(image):
    """
    Apply Anscombe transform to stabilize Poisson noise variance.
    
    Formula: f(x) = 2 * sqrt(x + 3/8)
    
    This transforms Poisson noise → approximately Gaussian noise
    """
    image_float = image.astype(np.float64)
    transformed = 2 * np.sqrt(image_float + 3.0/8.0)
    return transformed


def inverse_anscombe_transform(transformed_image):
    """
    Apply inverse Anscombe transform to recover the image.
    
    Formula: f^(-1)(y) = (y/2)^2 - 3/8
    """
    recovered = (transformed_image / 2.0) ** 2 - 3.0/8.0
    recovered = np.maximum(recovered, 0)  # Ensure non-negative
    recovered = np.clip(recovered, 0, 255).astype(np.uint8)
    return recovered


def denoise_with_anscombe(noisy_image, filter_type='gaussian', kernel_size=5):
    """
    Denoise image using the Anscombe transform method.
    
    Process:
    1. Apply Anscombe transform (stabilize variance)
    2. Apply denoising filter in transformed domain
    3. Apply inverse Anscombe transform (recover image)
    
    Parameters:
    -----------
    noisy_image : numpy array
        Noisy input image
    filter_type : str
        'gaussian', 'median', or 'bilateral'
    kernel_size : int
        Filter kernel size (must be odd number)
    
    Returns:
    --------
    numpy array : Denoised image
    """
    # Handle color images by processing each channel separately
    if len(noisy_image.shape) == 3:
        channels = cv2.split(noisy_image)
        denoised_channels = []
        
        for channel in channels:
            # Step 1: Apply Anscombe transform
            transformed = anscombe_transform(channel)
            
            # Step 2: Apply denoising filter
            if filter_type == 'gaussian':
                filtered = cv2.GaussianBlur(transformed, (kernel_size, kernel_size), 0)
            elif filter_type == 'median':
                filtered = cv2.medianBlur(transformed.astype(np.float32), kernel_size).astype(np.float64)
            elif filter_type == 'bilateral':
                filtered = cv2.bilateralFilter(transformed.astype(np.float32), kernel_size, 75, 75).astype(np.float64)
            else:
                filtered = transformed
            
            # Step 3: Apply inverse Anscombe transform
            denoised_channel = inverse_anscombe_transform(filtered)
            denoised_channels.append(denoised_channel)
        
        denoised = cv2.merge(denoised_channels)
    else:
        # Grayscale image
        transformed = anscombe_transform(noisy_image)
        
        if filter_type == 'gaussian':
            filtered = cv2.GaussianBlur(transformed, (kernel_size, kernel_size), 0)
        elif filter_type == 'median':
            filtered = cv2.medianBlur(transformed.astype(np.float32), kernel_size).astype(np.float64)
        elif filter_type == 'bilateral':
            filtered = cv2.bilateralFilter(transformed.astype(np.float32), kernel_size, 75, 75).astype(np.float64)
        else:
            filtered = transformed
        
        denoised = inverse_anscombe_transform(filtered)
    
    return denoised

print("✓ Anscombe transform functions defined")

✓ Anscombe transform functions defined


## 5. Define Image Quality Metric Functions

These functions measure how well the denoising worked:

- **PSNR** (Peak Signal-to-Noise Ratio): Higher is better (measures pixel accuracy)
- **SSIM** (Structural Similarity): Closer to 1 is better (measures perceived quality)
- **MSE** (Mean Squared Error): Lower is better (pixel-level error)
- **Entropy**: Measures information content
- **Sharpness**: Uses Laplacian variance to measure edge clarity
- **Spatial Frequency**: Measures image detail
- **Dynamic Range**: Measures contrast (max - min intensity)

In [13]:
def calculate_entropy(image):
    """Calculate entropy (information content) of image."""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    hist, _ = np.histogram(gray.flatten(), bins=256, range=(0, 256))
    hist = hist / hist.sum()
    hist = hist[hist > 0]
    return entropy(hist, base=2)


def calculate_sharpness(image):
    """Calculate image sharpness using Laplacian variance."""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    return laplacian.var()


def calculate_spatial_frequency(image):
    """Calculate spatial frequency (measures image detail)."""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    gray = gray.astype(np.float64)
    RF = np.sqrt(np.mean(np.diff(gray, axis=1) ** 2))  # Row frequency
    CF = np.sqrt(np.mean(np.diff(gray, axis=0) ** 2))  # Column frequency
    SF = np.sqrt(RF ** 2 + CF ** 2)
    return SF


def calculate_dynamic_range(image):
    """Calculate dynamic range (max - min intensity)."""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    return np.max(gray) - np.min(gray)


def calculate_noise_variance(original, noisy):
    """Calculate variance of the noise (difference between images)."""
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        noisy_gray = cv2.cvtColor(noisy, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        noisy_gray = noisy

    noise = noisy_gray.astype(np.float64) - original_gray.astype(np.float64)
    return np.var(noise)


def calculate_all_metrics(original, processed, prefix=""):
    """
    Calculate all quality metrics comparing original to processed image.
    
    Parameters:
    -----------
    original : numpy array
        Original clean image
    processed : numpy array
        Noisy or denoised image to compare
    prefix : str
        Prefix for metric names (e.g., 'Noisy_' or 'Denoised_')
    
    Returns:
    --------
    dict : Dictionary containing all calculated metrics
    """
    # Convert to grayscale for metric calculation
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        processed_gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        processed_gray = processed

    # Calculate standard metrics
    psnr_value = psnr(original_gray, processed_gray, data_range=255)
    ssim_value = ssim(original_gray, processed_gray, data_range=255)
    mse_value = np.mean((original_gray.astype(np.float64) - processed_gray.astype(np.float64)) ** 2)
    
    # Calculate entropy difference
    entropy_orig = calculate_entropy(original)
    entropy_processed = calculate_entropy(processed)
    entropy_diff = abs(entropy_processed - entropy_orig)
    
    # Calculate additional metrics
    noise_var = calculate_noise_variance(original, processed)
    sharpness_value = calculate_sharpness(processed)
    spatial_freq = calculate_spatial_frequency(processed)
    dynamic_range = calculate_dynamic_range(processed)

    return {
        f"{prefix}PSNR": psnr_value,
        f"{prefix}SSIM": ssim_value,
        f"{prefix}MSE": mse_value,
        f"{prefix}Entropy_Diff": entropy_diff,
        f"{prefix}Noise_Variance": noise_var,
        f"{prefix}Sharpness": sharpness_value,
        f"{prefix}Spatial_Freq": spatial_freq,
        f"{prefix}Dynamic_Range": dynamic_range
    }

print("✓ All metric calculation functions defined")

✓ All metric calculation functions defined


## 6. Configure Processing Parameters

Define the noise levels and denoising filters to test:

**Noise Scales:**
- 0.1 = Very high noise (low photon count)
- 0.5 = High noise
- 1.0 = Standard Poisson noise
- 2.0 = Moderate noise
- 5.0 = Low noise (high photon count)

**Denoising Filters:**
- Gaussian blur (kernel sizes 3, 5, 7)
- Median filter (kernel sizes 3, 5)
- Bilateral filter (kernel size 5)

This creates **30 total combinations** to test (5 noise levels × 6 filters).

In [14]:
# ============ NOISE LEVEL CONFIGURATIONS ============
POISSON_PARAMS = [
    {"scale": 0.1, "name": "scale_0.1"},   # Very high noise
    {"scale": 0.5, "name": "scale_0.5"},   # High noise
    {"scale": 1.0, "name": "scale_1.0"},   # Standard Poisson
    {"scale": 2.0, "name": "scale_2.0"},   # Moderate noise
    {"scale": 5.0, "name": "scale_5.0"},   # Low noise
]

# ============ DENOISING FILTER CONFIGURATIONS ============
ANSCOMBE_FILTERS = [
    {"filter_type": "gaussian", "kernel_size": 3, "name": "gaussian_k3"},
    {"filter_type": "gaussian", "kernel_size": 5, "name": "gaussian_k5"},
    {"filter_type": "gaussian", "kernel_size": 7, "name": "gaussian_k7"},
    {"filter_type": "median", "kernel_size": 3, "name": "median_k3"},
    {"filter_type": "median", "kernel_size": 5, "name": "median_k5"},
    {"filter_type": "bilateral", "kernel_size": 5, "name": "bilateral_k5"},
]

print(f"✓ Configured {len(POISSON_PARAMS)} noise levels")
print(f"✓ Configured {len(ANSCOMBE_FILTERS)} denoising filters")
print(f"✓ Total combinations to test: {len(POISSON_PARAMS) * len(ANSCOMBE_FILTERS)}")

✓ Configured 5 noise levels
✓ Configured 6 denoising filters
✓ Total combinations to test: 30


## 7. Setup Output Directories and Load Images

This cell:
1. Creates the output directory structure (if it doesn't exist)
2. Scans the input folder for image files
3. Separates color and black & white images
4. Selects 5 sample images for detailed comparison visualizations

In [15]:
# ============ CREATE OUTPUT DIRECTORIES ============
metrics_root = os.path.join(output_base, "Poisson_Anscombe_Metric")
noised_root = os.path.join(output_base, "Poisson_Noised_Images")
sample_root = os.path.join(output_base, "Poisson_Anscombe_Sample")
graph_root = os.path.join(output_base, "graph/poisson")

# Create all directories (checking if they already exist)
for directory in [metrics_root, noised_root, sample_root, graph_root]:
    Path(directory).mkdir(parents=True, exist_ok=True)
    print(f"✓ Directory ready: {directory}")

# ============ LOAD IMAGE FILES ============
# Get all image files from input directory
all_files = [f for f in os.listdir(base_path)
             if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))
             and os.path.isfile(os.path.join(base_path, f))]

# Separate color and black & white images
color_files = []
bw_files = []

for filename in all_files:
    # Check if filename indicates black & white
    if filename[-6:-4] == 'bw' or filename.lower().endswith('_bw.jpg') or filename.lower().endswith('_bw.png'):
        bw_files.append(filename)
    else:
        color_files.append(filename)

# ============ SELECT SAMPLE IMAGES ============
# Choose 5 images for detailed comparison visualizations
sample_images = []
if len(all_files) >= 5:
    # Try to get a mix of color and B&W
    sample_color = color_files[:3] if len(color_files) >= 3 else color_files
    sample_bw = bw_files[:2] if len(bw_files) >= 2 else bw_files
    sample_images = sample_color + sample_bw
    # If we don't have 5 yet, just take first 5 overall
    if len(sample_images) < 5:
        sample_images = all_files[:5]
else:
    sample_images = all_files

# ============ SUMMARY ============
print(f"\n{'='*70}")
print(f"IMAGE INVENTORY")
print(f"{'='*70}")
print(f"Color images found: {len(color_files)}")
print(f"B&W images found: {len(bw_files)}")
print(f"Total images: {len(all_files)}")
print(f"Sample images selected: {len(sample_images)}")
print(f"{'='*70}")

✓ Directory ready: data_results/data\Poisson_Anscombe_Metric
✓ Directory ready: data_results/data\Poisson_Noised_Images
✓ Directory ready: data_results/data\Poisson_Anscombe_Sample
✓ Directory ready: data_results/data\graph/poisson

IMAGE INVENTORY
Color images found: 80
B&W images found: 80
Total images: 160
Sample images selected: 5


## 8. Main Processing Loop: Add Noise and Denoise

This is the main processing cell that:

**For each noise level (5 levels):**
1. Adds Poisson noise to all images
2. Saves noisy images to disk
3. Calculates quality metrics for noisy images
4. Saves metrics to CSV files

**For each denoising filter (6 filters):**
1. Applies Anscombe transform denoising
2. Calculates quality metrics and improvements
3. Saves denoised metrics to CSV files
4. Stores sample images for visualization

**Total iterations:** 5 noise levels × 6 filters = 30 combinations

This cell may take several minutes depending on the number of images.

In [16]:
# ============ MAIN PROCESSING LOOP ============

# Dictionary to store sample images for visualization later
sample_data = {}

for poisson_config in POISSON_PARAMS:
    param_name = poisson_config["name"]
    scale = poisson_config["scale"]

    print(f"\n{'='*70}")
    print(f"Processing Noise Level: {param_name} (scale={scale})")
    print(f"{'='*70}")

    # Create output directory for this noise level
    noise_output_dir = os.path.join(noised_root, param_name)
    os.makedirs(noise_output_dir, exist_ok=True)

    # Store noisy images in memory for denoising
    noisy_images_dict = {}
    
    # Separate metric storage for color vs B&W
    noisy_metrics_color = []
    noisy_metrics_bw = []

    # -------- STEP 1: Add Poisson Noise --------
    print(f"\n  [1/2] Adding Poisson noise to images...")
    for filename in tqdm(all_files, desc=f"    Adding noise ({param_name})"):
        filepath = os.path.join(base_path, filename)
        img = cv2.imread(filepath)

        if img is None:
            continue

        # Add Poisson noise
        noisy_img = add_poisson_noise(img, scale=scale)

        # Save noisy image to disk
        output_filename = f"noisy_{filename}"
        output_path = os.path.join(noise_output_dir, output_filename)
        cv2.imwrite(output_path, noisy_img)

        # Store in memory for denoising
        noisy_images_dict[filename] = noisy_img

        # Store for sample visualization
        if filename in sample_images:
            if filename not in sample_data:
                sample_data[filename] = {'original': img, 'noisy': {}, 'denoised': {}}
            sample_data[filename]['noisy'][param_name] = noisy_img

        # Calculate quality metrics
        noisy_metrics = calculate_all_metrics(img, noisy_img, prefix="Noisy_")
        noisy_metrics["Name"] = output_filename
        noisy_metrics["Noise_Level"] = param_name
        noisy_metrics["Scale"] = scale

        # Separate by color vs B&W
        if filename[-6:-4] == 'bw' or filename.lower().endswith('_bw.jpg') or filename.lower().endswith('_bw.png'):
            noisy_metrics_bw.append(noisy_metrics)
        else:
            noisy_metrics_color.append(noisy_metrics)

    # Save noisy metrics to CSV
    column_order = ["Name", "Noise_Level", "Scale", "Noisy_PSNR", "Noisy_SSIM", "Noisy_MSE", "Noisy_Entropy_Diff",
                    "Noisy_Noise_Variance", "Noisy_Sharpness", "Noisy_Spatial_Freq", "Noisy_Dynamic_Range"]
    
    if noisy_metrics_color:
        df_color = pd.DataFrame(noisy_metrics_color)[column_order]
        csv_path_color = os.path.join(metrics_root, f"noisy_{param_name}.csv")
        df_color.to_csv(csv_path_color, index=False)
        print(f"  ✓ Saved color metrics: {csv_path_color}")

    if noisy_metrics_bw:
        df_bw = pd.DataFrame(noisy_metrics_bw)[column_order]
        csv_path_bw = os.path.join(metrics_root, f"noisy_{param_name}_bw.csv")
        df_bw.to_csv(csv_path_bw, index=False)
        print(f"  ✓ Saved B&W metrics: {csv_path_bw}")

    # -------- STEP 2: Apply Anscombe Denoising --------
    for filter_config in ANSCOMBE_FILTERS:
        filter_name = filter_config["name"]
        filter_type = filter_config["filter_type"]
        kernel_size = filter_config["kernel_size"]

        print(f"\n  [2/2] Denoising with {filter_name}...")

        # Separate metric storage
        denoised_metrics_color = []
        denoised_metrics_bw = []

        for filename in tqdm(all_files, desc=f"    Denoising ({filter_name})"):
            filepath = os.path.join(base_path, filename)
            img = cv2.imread(filepath)

            if img is None or filename not in noisy_images_dict:
                continue

            noisy_img = noisy_images_dict[filename]

            # Apply Anscombe transform denoising
            denoised_img = denoise_with_anscombe(noisy_img, filter_type=filter_type, kernel_size=kernel_size)

            # Store for sample visualization
            if filename in sample_images:
                if param_name not in sample_data[filename]['denoised']:
                    sample_data[filename]['denoised'][param_name] = {}
                sample_data[filename]['denoised'][param_name][filter_name] = denoised_img

            # Calculate quality metrics
            denoised_metrics = calculate_all_metrics(img, denoised_img, prefix="Denoised_")
            
            # Calculate improvement over noisy
            noisy_psnr = calculate_all_metrics(img, noisy_img, prefix="")["PSNR"]
            noisy_ssim = calculate_all_metrics(img, noisy_img, prefix="")["SSIM"]
            noisy_mse = calculate_all_metrics(img, noisy_img, prefix="")["MSE"]
            
            improvement_metrics = {
                "PSNR_Improvement": denoised_metrics["Denoised_PSNR"] - noisy_psnr,
                "SSIM_Improvement": denoised_metrics["Denoised_SSIM"] - noisy_ssim,
                "MSE_Reduction": noisy_mse - denoised_metrics["Denoised_MSE"],
            }
            
            denoised_metrics["Name"] = f"denoised_{filename}"
            denoised_metrics["Noise_Level"] = param_name
            denoised_metrics["Scale"] = scale
            denoised_metrics["Filter"] = filter_name
            denoised_metrics.update(improvement_metrics)

            # Separate by color vs B&W
            if filename[-6:-4] == 'bw' or filename.lower().endswith('_bw.jpg') or filename.lower().endswith('_bw.png'):
                denoised_metrics_bw.append(denoised_metrics)
            else:
                denoised_metrics_color.append(denoised_metrics)

        # Save denoised metrics to CSV
        denoised_column_order = ["Name", "Noise_Level", "Scale", "Filter", 
                                "Denoised_PSNR", "Denoised_SSIM", "Denoised_MSE", 
                                "Denoised_Entropy_Diff", "Denoised_Noise_Variance", "Denoised_Sharpness", 
                                "Denoised_Spatial_Freq", "Denoised_Dynamic_Range",
                                "PSNR_Improvement", "SSIM_Improvement", "MSE_Reduction"]
        
        if denoised_metrics_color:
            df_color = pd.DataFrame(denoised_metrics_color)[denoised_column_order]
            csv_path_color = os.path.join(metrics_root, f"denoised_{param_name}_{filter_name}.csv")
            df_color.to_csv(csv_path_color, index=False)
            print(f"    ✓ Saved color metrics: {csv_path_color}")

        if denoised_metrics_bw:
            df_bw = pd.DataFrame(denoised_metrics_bw)[denoised_column_order]
            csv_path_bw = os.path.join(metrics_root, f"denoised_{param_name}_{filter_name}_bw.csv")
            df_bw.to_csv(csv_path_bw, index=False)
            print(f"    ✓ Saved B&W metrics: {csv_path_bw}")

print("\n✓ All processing complete!")
print(f"✓ Sample data collected for {len(sample_data)} images")


Processing Noise Level: scale_0.1 (scale=0.1)

  [1/2] Adding Poisson noise to images...


    Adding noise (scale_0.1): 100%|██████████| 160/160 [00:07<00:00, 21.38it/s] 


  ✓ Saved color metrics: data_results/data\Poisson_Anscombe_Metric\noisy_scale_0.1.csv
  ✓ Saved B&W metrics: data_results/data\Poisson_Anscombe_Metric\noisy_scale_0.1_bw.csv

  [2/2] Denoising with gaussian_k3...


    Denoising (gaussian_k3): 100%|██████████| 160/160 [00:16<00:00,  9.78it/s]


    ✓ Saved color metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_0.1_gaussian_k3.csv
    ✓ Saved B&W metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_0.1_gaussian_k3_bw.csv

  [2/2] Denoising with gaussian_k5...


...

    ✓ Saved color metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_5.0_median_k3.csv
    ✓ Saved B&W metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_5.0_median_k3_bw.csv

  [2/2] Denoising with median_k5...


    Denoising (median_k5): 100%|██████████| 160/160 [00:40<00:00,  3.97it/s]


    ✓ Saved color metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_5.0_median_k5.csv
    ✓ Saved B&W metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_5.0_median_k5_bw.csv

  [2/2] Denoising with bilateral_k5...


    Denoising (bilateral_k5): 100%|██████████| 160/160 [00:16<00:00,  9.60it/s]

    ✓ Saved color metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_5.0_bilateral_k5.csv
    ✓ Saved B&W metrics: data_results/data\Poisson_Anscombe_Metric\denoised_scale_5.0_bilateral_k5_bw.csv

✓ All processing complete!
✓ Sample data collected for 5 images


## 9. Create Sample Image Comparisons

This cell creates side-by-side visual comparisons showing:
- Original image
- Noisy version (for each scale level)
- All 6 denoised versions (using different filters)

Saves comparison grids as PNG files for visual inspection of denoising effectiveness.

In [17]:
# ============ CREATE SAMPLE COMPARISONS ============
print(f"\n{'='*70}")
print("CREATING SAMPLE IMAGE COMPARISONS")
print(f"{'='*70}\n")

for filename, data in sample_data.items():
    print(f"Processing sample: {filename}")
    
    # Create a comparison for each noise level
    for noise_level in POISSON_PARAMS:
        param_name = noise_level["name"]
        scale = noise_level["scale"]
        
        if param_name not in data['noisy']:
            continue
        
        # Get images
        original = data['original']
        noisy = data['noisy'][param_name]
        
        # Create figure: 2 rows × 4 columns
        fig, axes = plt.subplots(2, 4, figsize=(20, 10))
        fig.suptitle(f'Sample: {filename} | Noise Level: {param_name} (scale={scale})', 
                     fontsize=16, fontweight='bold')
        
        # Convert BGR to RGB for matplotlib
        original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
        noisy_rgb = cv2.cvtColor(noisy, cv2.COLOR_BGR2RGB)
        
        # Position [0,0]: Original image
        axes[0, 0].imshow(original_rgb)
        axes[0, 0].set_title('Original', fontsize=12, fontweight='bold')
        axes[0, 0].axis('off')
        
        # Position [0,1]: Noisy image
        axes[0, 1].imshow(noisy_rgb)
        axes[0, 1].set_title(f'Noisy (scale={scale})', fontsize=12, fontweight='bold')
        axes[0, 1].axis('off')
        
        # Positions [0,2], [0,3], [1,0], [1,1], [1,2], [1,3]: Denoised versions
        positions = [(0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)]
        
        for idx, filter_config in enumerate(ANSCOMBE_FILTERS):
            if idx >= 6:  # Only show first 6 filters
                break
                
            filter_name = filter_config["name"]
            
            if param_name in data['denoised'] and filter_name in data['denoised'][param_name]:
                denoised = data['denoised'][param_name][filter_name]
                denoised_rgb = cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB)
                
                row, col = positions[idx]
                axes[row, col].imshow(denoised_rgb)
                axes[row, col].set_title(f'Denoised: {filter_name}', fontsize=10)
                axes[row, col].axis('off')
        
        plt.tight_layout()
        
        # Save comparison image
        sample_filename = os.path.splitext(filename)[0]
        output_path = os.path.join(sample_root, f"comparison_{sample_filename}_{param_name}.png")
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        print(f"  ✓ Saved: comparison_{sample_filename}_{param_name}.png")

print(f"\n✓ All sample comparisons saved to: {sample_root}")


CREATING SAMPLE IMAGE COMPARISONS

Processing sample: BSDS300_10_46076.jpg
  ✓ Saved: comparison_BSDS300_10_46076_scale_0.1.png
  ✓ Saved: comparison_BSDS300_10_46076_scale_0.5.png
  ✓ Saved: comparison_BSDS300_10_46076_scale_1.0.png
  ✓ Saved: comparison_BSDS300_10_46076_scale_2.0.png
  ✓ Saved: comparison_BSDS300_10_46076_scale_5.0.png
Processing sample: BSDS300_10_46076_bw.jpg
  ✓ Saved: comparison_BSDS300_10_46076_bw_scale_0.1.png
  ✓ Saved: comparison_BSDS300_10_46076_bw_scale_0.5.png
  ✓ Saved: comparison_BSDS300_10_46076_bw_scale_1.0.png
  ✓ Saved: comparison_BSDS300_10_46076_bw_scale_2.0.png
  ✓ Saved: comparison_BSDS300_10_46076_bw_scale_5.0.png
Processing sample: BSDS300_11_66039.jpg
  ✓ Saved: comparison_BSDS300_11_66039_scale_0.1.png
  ✓ Saved: comparison_BSDS300_11_66039_scale_0.5.png
  ✓ Saved: comparison_BSDS300_11_66039_scale_1.0.png
  ✓ Saved: comparison_BSDS300_11_66039_scale_2.0.png
  ✓ Saved: comparison_BSDS300_11_66039_scale_5.0.png
Processing sample: BSDS300_11_6

## 10. Create Metric Distribution Visualizations

This cell generates histogram visualizations showing how denoising quality varies across different noise levels.

**Creates 4 visualizations (one for each metric):**
1. **PSNR** - Higher values indicate better reconstruction
2. **SSIM** - Values closer to 1.0 indicate better perceptual quality
3. **MSE** - Lower values indicate less error
4. **Entropy Difference** - Measures information preservation

**Each visualization includes:**
- Overlaid histograms by noise level (left plot)
- Box plots showing distribution statistics (right plot)
- Summary statistics printed to console

In [ ]:
# ============ CREATE METRIC HISTOGRAMS ============
print(f"\n{'='*70}")
print("CREATING METRIC DISTRIBUTION VISUALIZATIONS")
print(f"{'='*70}\n")

# Load all denoised CSV files
all_denoised_data = []

for csv_file in os.listdir(metrics_root):
    if csv_file.startswith('denoised_') and csv_file.endswith('.csv'):
        csv_path = os.path.join(metrics_root, csv_file)
        df = pd.read_csv(csv_path)
        all_denoised_data.append(df)

if all_denoised_data:
    # Combine all data into single DataFrame
    combined_df = pd.concat(all_denoised_data, ignore_index=True)
    
    print(f"Loaded {len(all_denoised_data)} CSV files")
    print(f"Total records: {len(combined_df)}")
    print(f"Noise levels: {sorted(combined_df['Noise_Level'].unique())}")
    print(f"Filters tested: {sorted(combined_df['Filter'].unique())}")
    
    # Define metrics to visualize
    metrics_to_plot = [
        ('Denoised_PSNR', 'PSNR (dB)', 'Peak Signal-to-Noise Ratio'),
        ('Denoised_SSIM', 'SSIM', 'Structural Similarity Index'),
        ('Denoised_MSE', 'MSE', 'Mean Squared Error'),
        ('Denoised_Entropy_Diff', 'Entropy Difference', 'Entropy Difference from Original')
    ]
    
    # Get sorted noise levels and unique filter types
    noise_levels_sorted = sorted(combined_df['Noise_Level'].unique(), 
                                 key=lambda x: float(x.split('_')[1]))
    
    # Group filters by type (extract base filter name)
    filter_types = {}
    for filter_name in combined_df['Filter'].unique():
        base_type = filter_name.rsplit('_', 1)[0]  # e.g., 'gaussian_k3' -> 'gaussian'
        if base_type not in filter_types:
            filter_types[base_type] = []
        filter_types[base_type].append(filter_name)
    
    # Sort filters within each type by kernel size
    for base_type in filter_types:
        filter_types[base_type] = sorted(filter_types[base_type], 
                                        key=lambda x: int(x.split('_k')[1]))
    
    # Create visualization for each metric
    for metric_col, xlabel, title in metrics_to_plot:
        if metric_col not in combined_df.columns:
            print(f"  ⚠ Skipping {metric_col} - column not found")
            continue
        
        print(f"\nCreating visualizations for {title}...")
        
        # ===== CREATE GRID: Noise Levels (rows) × Filter Types (columns) =====
        n_noise_levels = len(noise_levels_sorted)
        n_filter_types = len(filter_types)
        
        fig, axes = plt.subplots(n_noise_levels, n_filter_types, 
                                figsize=(6*n_filter_types, 4*n_noise_levels))
        fig.suptitle(f'{title} Distribution by Noise Level and Filter Type', 
                     fontsize=18, fontweight='bold', y=0.995)
        
        # Ensure axes is 2D even if only 1 row or column
        if n_noise_levels == 1 and n_filter_types == 1:
            axes = np.array([[axes]])
        elif n_noise_levels == 1:
            axes = axes.reshape(1, -1)
        elif n_filter_types == 1:
            axes = axes.reshape(-1, 1)
        
        # Create color map for kernel sizes
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        
        # Plot each combination of noise level and filter type
        for row_idx, noise_level in enumerate(noise_levels_sorted):
            for col_idx, (base_type, filter_list) in enumerate(sorted(filter_types.items())):
                ax = axes[row_idx, col_idx]
                
                # Plot histogram for each kernel size in this filter type
                for filter_idx, filter_name in enumerate(filter_list):
                    data_subset = combined_df[
                        (combined_df['Noise_Level'] == noise_level) & 
                        (combined_df['Filter'] == filter_name)
                    ][metric_col]
                    
                    if len(data_subset) > 0:
                        kernel_size = filter_name.split('_k')[1]
                        ax.hist(data_subset, alpha=0.6, bins=15, 
                               color=colors[filter_idx % len(colors)],
                               label=f'kernel={kernel_size}', 
                               edgecolor='black', linewidth=0.5)
                
                # Formatting
                ax.set_xlabel(xlabel, fontsize=9)
                ax.set_ylabel('Frequency', fontsize=9)
                ax.grid(True, alpha=0.3)
                
                # Title for top row only
                if row_idx == 0:
                    ax.set_title(f'{base_type.capitalize()} Filter', 
                                fontsize=11, fontweight='bold')
                
                # Y-axis label for first column only
                if col_idx == 0:
                    ax.text(-0.3, 0.5, f'{noise_level}\n(scale={noise_level.split("_")[1]})', 
                           transform=ax.transAxes, fontsize=10, fontweight='bold',
                           verticalalignment='center', rotation=90)
                
                # Add legend if there are multiple kernel sizes
                if len(filter_list) > 1:
                    ax.legend(fontsize=8, loc='upper right')
        
        plt.tight_layout()
        
        # Save figure
        output_filename = f"histogram_{metric_col.lower()}_detailed.png"
        output_path = os.path.join(graph_root, output_filename)
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        print(f"  ✓ Saved: {output_filename}")
    
    # Print summary statistics organized by noise level and filter
    print("\n" + "="*70)
    print("SUMMARY STATISTICS")
    print("="*70)
    
    for metric_col, _, title in metrics_to_plot:
        if metric_col in combined_df.columns:
            print(f"\n{title}:")
            print("-" * 70)
            for noise_level in noise_levels_sorted:
                print(f"\n  Noise Level: {noise_level}")
                subset = combined_df[combined_df['Noise_Level'] == noise_level]
                summary = subset.groupby('Filter')[metric_col].agg(['mean', 'std', 'min', 'max'])
                print(summary.to_string())
    
    print(f"\n{'='*70}")
    print(f"✓ All visualizations saved to: {graph_root}")
    print(f"{'='*70}")
    
else:
    print("⚠ No denoised CSV files found. Run the main processing loop first.")


CREATING METRIC DISTRIBUTION VISUALIZATIONS

Loaded 60 CSV files
Total records: 4800
Noise levels: ['scale_0.1', 'scale_0.5', 'scale_1.0', 'scale_2.0', 'scale_5.0']
Filters tested: ['bilateral_k5', 'gaussian_k3', 'gaussian_k5', 'gaussian_k7', 'median_k3', 'median_k5']


C:\Users\ashto\AppData\Local\Temp\ipykernel_46504\4094848411.py:59: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = axes[1].boxplot(box_data, labels=noise_levels_sorted, patch_artist=True)


  ✓ Saved: histogram_denoised_psnr.png


C:\Users\ashto\AppData\Local\Temp\ipykernel_46504\4094848411.py:59: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = axes[1].boxplot(box_data, labels=noise_levels_sorted, patch_artist=True)


  ✓ Saved: histogram_denoised_ssim.png


C:\Users\ashto\AppData\Local\Temp\ipykernel_46504\4094848411.py:59: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = axes[1].boxplot(box_data, labels=noise_levels_sorted, patch_artist=True)


  ✓ Saved: histogram_denoised_mse.png


C:\Users\ashto\AppData\Local\Temp\ipykernel_46504\4094848411.py:59: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = axes[1].boxplot(box_data, labels=noise_levels_sorted, patch_artist=True)


  ✓ Saved: histogram_denoised_entropy_diff.png

SUMMARY STATISTICS BY NOISE LEVEL

Peak Signal-to-Noise Ratio:
                  mean       std        min        max
Noise_Level                                           
scale_0.1    28.229861  6.646678  16.361623  45.389065
scale_0.5    28.047368  6.393474  16.395674  43.604111
scale_1.0    27.860915  6.167319  16.423241  42.622397
scale_2.0    27.492655  5.791591  16.442609  41.183743
scale_5.0    26.637475  5.070142  16.447491  38.392240

Structural Similarity Index:
                 mean       std       min       max
Noise_Level                                        
scale_0.1    0.875551  0.107478  0.436757  0.991680
scale_0.5    0.868449  0.106629  0.433663  0.987597
scale_1.0    0.860851  0.106615  0.429498  0.983995
scale_2.0    0.847474  0.108176  0.423658  0.979428
scale_5.0    0.817605  0.117607  0.404693  0.972868

Mean Squared Error:
                   mean         std       min          max
Noise_Level                   

## 11. Final Summary

Displays a summary of all processing completed and output locations.

In [19]:
# ============ FINAL SUMMARY ============
print(f"\n{'='*70}")
print(f"PROCESSING COMPLETE")
print(f"{'='*70}")
print(f"\nConfigurations tested:")
print(f"  • Poisson noise levels: {len(POISSON_PARAMS)}")
print(f"  • Anscombe denoising filters: {len(ANSCOMBE_FILTERS)}")
print(f"  • Total combinations: {len(POISSON_PARAMS) * len(ANSCOMBE_FILTERS)}")
print(f"\nImages processed:")
print(f"  • Color images: {len(color_files)}")
print(f"  • B&W images: {len(bw_files)}")
print(f"  • Total images: {len(all_files)}")
print(f"  • Sample images: {len(sample_images)}")
print(f"\nOutput locations:")
print(f" Metrics (CSV files): {metrics_root}")
print(f" Noisy images: {noised_root}")
print(f" Sample comparisons: {sample_root}")
print(f" Visualizations: {graph_root}")
print(f"\nCSV files created:")
print(f"  • Noisy metrics: {len(POISSON_PARAMS) * 2} files (color + B&W)")
print(f"  • Denoised metrics: {len(POISSON_PARAMS) * len(ANSCOMBE_FILTERS) * 2} files")
print(f"\n✓ All tasks completed successfully!")


PROCESSING COMPLETE

Configurations tested:
  • Poisson noise levels: 5
  • Anscombe denoising filters: 6
  • Total combinations: 30

Images processed:
  • Color images: 80
  • B&W images: 80
  • Total images: 160
  • Sample images: 5

Output locations:
 Metrics (CSV files): data_results/data\Poisson_Anscombe_Metric
 Noisy images: data_results/data\Poisson_Noised_Images
 Sample comparisons: data_results/data\Poisson_Anscombe_Sample
 Visualizations: data_results/data\graph/poisson

CSV files created:
  • Noisy metrics: 10 files (color + B&W)
  • Denoised metrics: 60 files

✓ All tasks completed successfully!
